In [ ]:
#!pip install torch --upgrade
!pip install torchvision --upgrade

In [ ]:
from ultralytics import YOLO
import cv2
import math 
import gtts
from playsound import playsound
langu = 'en'

# start webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

focal_length = 5 #m

import time

timeout = 5
first_time = time.time()
last_time = first_time

def distance_finder(real_face_width, face_width_in_frame, focal_length = 5):
    distance = (real_face_width * focal_length) / face_width_in_frame
    return distance

def location_finder(h, w, x1, x2):
    diffx1 = x2 - w
    diffx2 = x1 - w
    return diffx1, diffx2

# model
model = YOLO("yolo-Weights/yolov8n.pt")

# object classes
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]

classNamesWidth = {"person":41, "bicycle":25, "car":177, "motorbike":76, "aeroplane":550, 
                   "bus":244, "train":153, "truck":244, "boat":244,"traffic light":35, "fire hydrant":30, 
                   "stop sign":20, "parking meter":20, "bench":100, "bird":8, "cat":8,
              "dog":10, "horse":150, "sheep":12, "cow":18, "elephant":500, "bear":300, "zebra":130, "giraffe":400, 
              "backpack":30, "umbrella":40,"handbag":20, "tie":6, "suitcase":20, "frisbee":12, 
              "skis":36, "snowboard":36, "sports ball":17, "kite":20, "baseball bat":30,"baseball glove":20, 
              "skateboard":36, "surfboard":40, "tennis racket":30, "bottle":12, "wine glass":8, "cup":12,
              "fork":3, "knife":5, "spoon":3, "bowl":15, "banana":15, "apple":7, "sandwich":7, "orange":8, 
              "broccoli":10,"carrot":15, "hot dog":15, "pizza":20, "donut":8, "cake":15, "chair":50, 
              "sofa":100, "pottedplant":20, "bed":130,"diningtable":200, "toilet":41, "tvmonitor":70,
              "laptop":40, "mouse":20, "remote":10, "keyboard":30, "cell phone":15,
              "microwave":30, "oven":40, "toaster":30, "sink":50, "refrigerator":50, "book":10, 
              "clock":10, "vase":10, "scissors":10,"teddy bear":20, "hair drier":15, "toothbrush":10}

while True:
    success, img = cap.read()
    results = model(img, stream=True)

    # coordinates
    for r in results:
        boxes = r.boxes

        for box in boxes:
            # bounding box
            print(box.xyxy)
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values

            # put box in cam
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 4)
            # confidence
            confidence = math.ceil((box.conf[0]*100))/100
            print("Confidence --->",confidence)

            # class name
            cls = int(box.cls[0])
            print(box.cls[0])
            print("Class name -->", classNames[cls])
            if (x2-x1) >=0:
                dist = distance_finder(classNamesWidth[classNames[cls]], (x2-x1), focal_length)
            print(f'{100*(dist):2f}M')
            # object details
            org = [x1, y1]
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontScale = 1
            color = (255, 0, 0)
            thickness = 2

            cv2.putText(img, classNames[cls], org, font, fontScale, color, thickness)
            print(f"image shape = {img.shape}")
            new_time = time.time()
#             if  new_time - last_time > timeout:
#                 last_time = new_time
            dist = round(100*(dist),2)
            (h, w) = img.shape[:2]
            h, w = h//2, w//2
                
            if dist < 40:
                c1, c2 = location_finder(h, w, x1, x2)
                if c1 > 0 and c2 > 0 :
                    t1 = gtts.gTTS(text =f"{classNames[cls]} is {dist} meter away on the left side", lang = langu, slow = False)
                    t1.save('sound.mp3')
                    playsound('sound.mp3')
                elif c1 < 0 and c2 < 0 :
                    t1 = gtts.gTTS(text =f"{classNames[cls]} is {dist} meter away on the right side", lang = langu, slow = False)
                    t1.save('sound.mp3')
                    playsound('sound.mp3')
                else:
                    t1 = gtts.gTTS(text =f"{classNames[cls]} is {dist} meter away just ahead you", lang = langu, slow = False)
                    t1.save('sound.mp3')
                    playsound('sound.mp3')
                
                
    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 480x640 4 persons, 88.6ms
Speed: 2.3ms preprocess, 88.6ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)



tensor([[1.6579e-01, 0.0000e+00, 1.4533e+02, 4.7803e+02]])
Confidence ---> 0.69
tensor(0.)
Class name --> person
141.379310M
image shape = (480, 640, 3)
tensor([[297.9407, 182.3212, 620.6194, 479.7100]])
Confidence ---> 0.56
tensor(0.)
Class name --> person
63.467492M
image shape = (480, 640, 3)
tensor([[299.0769, 304.7193, 421.7803, 479.8549]])
Confidence ---> 0.37
tensor(0.)
Class name --> person
168.032787M
image shape = (480, 640, 3)
tensor([[395.3201, 183.6247, 602.9142, 479.3768]])
Confidence ---> 0.29
tensor(0.)
Class name --> person
99.033816M
image shape = (480, 640, 3)
tensor([[1.1368e-01, 0.0000e+00, 1.4310e+02, 4.7886e+02]])
Confidence ---> 0.77
tensor(0.)
Class name --> person
143.356643M
image shape = (480, 640, 3)
tensor([[393.1033, 183.5396, 598.8895, 479.5237]])
Confidence ---> 0.47
tensor(0.)
Class name --> person
100.000000M
image shape = (480, 640, 3)
tensor([[283.0022, 183.8320, 609.7921, 480.0000]])
Confidence ---> 0.43
tensor(0.)
Class name --> person
62.883436M


0: 480x640 3 persons, 1 dog, 116.9ms
Speed: 2.1ms preprocess, 116.9ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 2 persons, 1 couch, 79.2ms
Speed: 3.0ms preprocess, 79.2ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 2 persons, 1 couch, 71.6ms
Speed: 2.4ms preprocess, 71.6ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)



tensor([[213.0891, 185.8186, 586.0501, 479.7563]])
Confidence ---> 0.89
tensor(0.)
Class name --> person
54.959786M
image shape = (480, 640, 3)
tensor([[1.4287e-01, 2.2039e+02, 1.0588e+02, 4.7931e+02]])
Confidence ---> 0.73
tensor(0.)
Class name --> person
195.238095M
image shape = (480, 640, 3)
tensor([[ 75.5211, 261.0331, 271.9755, 479.0994]])
Confidence ---> 0.32
tensor(57.)
Class name --> sofa
255.102041M
image shape = (480, 640, 3)
tensor([[205.2458, 190.5802, 609.7488, 479.4764]])
Confidence ---> 0.86
tensor(0.)
Class name --> person
50.742574M
image shape = (480, 640, 3)
tensor([[1.5387e-01, 2.2033e+02, 1.0643e+02, 4.7934e+02]])
Confidence ---> 0.78
tensor(0.)
Class name --> person
193.396226M
image shape = (480, 640, 3)
tensor([[ 76.2491, 260.4347, 263.1934, 478.8158]])
Confidence ---> 0.42
tensor(57.)
Class name --> sofa
267.379679M
image shape = (480, 640, 3)


0: 480x640 2 persons, 1 couch, 77.2ms
Speed: 1.4ms preprocess, 77.2ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 2 persons, 1 couch, 72.4ms
Speed: 1.5ms preprocess, 72.4ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)



...

tensor([[241.8017,   4.8528, 612.5632, 415.2067]])
Confidence ---> 0.34
tensor(0.)
Class name --> person
55.256065M
image shape = (480, 640, 3)
tensor([[ 49.9091, 306.0014, 408.7947, 479.5756]])
Confidence ---> 0.28
tensor(39.)
Class name --> bottle
16.713092M
image shape = (480, 640, 3)


0: 480x640 1 bottle, 64.8ms
Speed: 1.2ms preprocess, 64.8ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)



tensor([[ 31.3792,  44.0084, 638.8768, 479.4745]])
Confidence ---> 0.92
tensor(0.)
Class name --> person
33.772652M
image shape = (480, 640, 3)


0: 480x640 1 person, 1 cell phone, 73.3ms
Speed: 2.6ms preprocess, 73.3ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)



tensor([[395.9564, 389.3561, 461.7952, 459.7338]])
Confidence ---> 0.6
tensor(67.)
Class name --> cell phone
113.636364M
image shape = (480, 640, 3)
tensor([[ 34.4249,  15.2671, 639.2867, 478.7509]])
Confidence ---> 0.95
tensor(0.)
Class name --> person
33.884298M
image shape = (480, 640, 3)
